In [1]:
import pandas as pd
import numpy as np
# wizualizacje
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PoissonRegressor, GammaRegressor
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, r2_score

from patsy import dmatrices
import statsmodels.api as sm

# ustawienie szerokiego ekranu wyświetlacza JNotebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# opcje wyświetlania w Pandas
# - maks. 55 kolumn
# - maks. 101 wierszy
# - liczby w notacji dziesiętnej z czterema zerami po przecinku
pd.set_option('display.max_columns', 55)
pd.set_option('display.max_rows', 101)
pd.set_option('display.float_format', lambda x: f"{x:.4f}")

# opcje formatowania wykresów matplotlib
# - etykiety osi: bold
# - tekst: bold
# - domyślny rozmiar fontu=14
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['font.size'] = '14'

# Wczytanie danych

In [2]:
# Zadanie 1

filepath = r"C:\Users\pnaumczyk\Documents\Dane\Python_modelowanie_crashCourse\train.csv"
cats_dow = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Weekend"]
dow_dtype = pd.api.types.CategoricalDtype(categories=cats_dow, ordered=True)

miesiac_zima = ["January", "February", "December"]
miesiac_wiosna = ["March", "April", "May"]
miesiac_lato = ["June", "July", "August"]
miesiac_jesien = ["September", "October", "November"]

cats_season = ["wiosna", "lato", "jesien", "zima"]
season_dtype =  pd.api.types.CategoricalDtype(categories=cats_season, ordered=True)

cat_cols = [
    'pora_dnia_przejazdu',
    'dzien_tygodnia_cat', 
    'stacja_start_cat',  
    'stacja_finisz_cat', 
    'rodzaj_pociagu_cat', 
    'pora_roku_cat'
]
num_cols = [
    'opoznienie_start',
    'czy_szkoda',
    'wyplata_szkoda'
]


df = (
    pd.read_csv(filepath, sep=';')
    
    .assign(
        **{
             k: lambda df_, col = k: pd.to_datetime(df_[col])
                for k in [
                    'Kiedy jechał*ś?'
                ]
        },
        **{ k: lambda df_, col = k: pd.to_numeric(df_[col], downcast='float')
                for k in [
                    'Koszt podróży:'
                ]
        },
        **{ k: lambda df_, col = k: pd.to_numeric(df_[col], downcast='integer')
                for k in [
                    'Liczba minut opóźnienia na starcie:', 'Liczba minut opóźnienia na mecie:'
                ]
        }
    )
    
    .astype({
        **{ k: 'string' 
               for k in [
                   'Stacja początkowa:', 'Stacja końcowa:', 'Jak wrażenia z podróży? :D (nieobowiązkowe)', 'Uwagi (nieobowiązkowe):'
               ]            
        },
        **{ k: 'category' 
               for k in [
                   'Rodzaj pociągu:', 'Pora dnia:'
               ]            
        }
    })
    
    .rename(columns={
        'Kiedy jechał*ś?'         : 'data_przejazdu',
        'Stacja początkowa:'      : 'stacja_start',
        'Stacja końcowa:'         : 'stacja_finisz',
        'Rodzaj pociągu:'         : 'rodzaj_pociagu',
        'Uwagi (nieobowiązkowe):' : 'uwagi',
        'Koszt podróży:'          : 'cena_biletu',
        'Pora dnia:'              : 'pora_dnia_przejazdu',
        'Liczba minut opóźnienia na starcie:' : 'opoznienie_start',
        'Liczba minut opóźnienia na mecie:'   : 'opoznienie_finisz',
        'Jak wrażenia z podróży? :D (nieobowiązkowe)' : 'wrazenia'
    })
    
    .drop(columns=['wrazenia', 'uwagi'])
    .drop(index=[30, 50])
    
    .assign(
        pora_dnia_przejazdu = lambda df_: np.select(
        [
            (df_.pora_dnia_przejazdu.isnull() & df_.stacja_start.str.contains('Legionowo')).astype(bool),
            (df_.pora_dnia_przejazdu.isnull() & df_.stacja_finisz.str.contains('Legionowo')).astype(bool),
            (df_.pora_dnia_przejazdu.isnull() & df_.stacja_finisz.str.contains('Gdask')).astype(bool)
        ],
        [
            'rano',
            'popołudnie',
            'popołudnie'            
        ],
        df_.pora_dnia_przejazdu
    ))
    
    .reset_index(drop=True)
    
    .assign(
        czy_szkoda = lambda df_: np.select(
            [
                df_.opoznienie_finisz.gt(10) & df_.opoznienie_finisz.le(40),
                df_.opoznienie_finisz.gt(40)
            ],
            [
                1,
                2
            ],
            0
        ),
        
        wyplata_szkoda = lambda df_: np.where(
            df_.opoznienie_finisz.gt(10),
            df_.cena_biletu * np.exp(df_.opoznienie_finisz.div(100)) * df_.czy_szkoda,
            0.0
        )

    )
    
    .sort_values(by='data_przejazdu')
    .reset_index(drop=True)
    
    .assign(
        dzien_tygodnia = lambda df_: df_.data_przejazdu.dt.day_name().astype('category'),
        dzien_tygodnia_cat = lambda df_: np.where(
            df_.dzien_tygodnia.eq('Sunday') | df_.dzien_tygodnia.eq('Saturday') | df_.dzien_tygodnia.eq('Monday'),
            'Weekend',
            df_.dzien_tygodnia
        ),
        miesiac = lambda df_: df_.data_przejazdu.dt.month_name().astype('category'),
        pora_roku = lambda df_: np.select(
            [
                df_.miesiac.isin(miesiac_zima),
                df_.miesiac.isin(miesiac_wiosna),
                df_.miesiac.isin(miesiac_lato),
                df_.miesiac.isin(miesiac_jesien)
            ],
            [
                "zima",
                "wiosna",
                "lato",
                "jesien"
            ],
            "brak_danych"
        ),        
        stacja_start_cat = lambda df_: np.select(
            [
                df_.stacja_start.str.contains("Warszawa").astype(bool),
                df_.stacja_start.str.contains("Legionowo").astype(bool),
                (df_.stacja_start.str.contains("Gdańsk") | df_.stacja_start.str.contains("Gdask")).astype(bool)
            ],
            [
                "Warszawa",
                "Legionowo",
                "Gdańsk"
                
            ],
            "Inne"
        ),
        stacja_finisz_cat = lambda df_: np.select(
            [
                df_.stacja_finisz.str.contains("Warszawa").astype(bool),
                df_.stacja_finisz.str.contains("Legionowo").astype(bool),
                (df_.stacja_finisz.str.contains("Gdańsk") | df_.stacja_finisz.str.contains("Gdask")).astype(bool)
            ],
            [
                "Warszawa",
                "Legionowo",
                "Gdańsk"
                
            ],
            "Inne"
        ),  
        rodzaj_pociagu_cat = lambda df_: np.where(df_.rodzaj_pociagu.eq("EIC"), '"Zwykły" InterCity', df_.rodzaj_pociagu)
    )
    
    .astype({
        **{ k: 'category' 
               for k in [
                   'stacja_start_cat', 'stacja_finisz_cat', 'rodzaj_pociagu_cat'
               ]            
        },      
    })
    
    .assign(
        dzien_tygodnia_cat = lambda df_: df_.dzien_tygodnia_cat.astype(dow_dtype),
        pora_roku_cat = lambda df_: df_.pora_roku.astype(season_dtype)
    )
    
    .drop(columns=[
        'data_przejazdu', 'stacja_start', 'stacja_finisz', 'rodzaj_pociagu', 'cena_biletu', 'dzien_tygodnia', 'miesiac', 'pora_roku', 'opoznienie_finisz'
    ])

    .reindex(columns = cat_cols + num_cols)

)
df

,pora_dnia_przejazdu,dzien_tygodnia_cat,stacja_start_cat,stacja_finisz_cat,rodzaj_pociagu_cat,pora_roku_cat,opoznienie_start,czy_szkoda,wyplata_szkoda
0,popołudnie,Thursday,Warszawa,Gdańsk,"""Zwykły"" InterCity",zima,0,1,99.7513
1,popołudnie,Tuesday,Gdańsk,Legionowo,"""Zwykły"" InterCity",zima,0,0,0.0000
2,rano,Wednesday,Legionowo,Warszawa,Podmiejski / Regionalne,zima,0,0,0.0000
3,popołudnie,Wednesday,Warszawa,Legionowo,Podmiejski / Regionalne,zima,0,0,0.0000
4,popołudnie,Thursday,Warszawa,Legionowo,Podmiejski / Regionalne,zima,3,0,0.0000
...,...,...,...,...,...,...,...,...,...
98,rano,Weekend,Gdańsk,Warszawa,Pendolino,wiosna,0,0,0.0000
99,popołudnie,Tuesday,Warszawa,Gdańsk,Pendolino,wiosna,0,0,0.0000
100,rano,Tuesday,Gdańsk,Warszawa,Pendolino,wiosna,26,1,247.1261
101,popołudnie,Thursday,Warszawa,Gdańsk,"""Zwykły"" InterCity",wiosna,0,0,0.0000


# Spis treści
1. [Podział na set testowy i treningowy](#train_test)
2. [Przekształcenia zmiennych kategorialnych](#encode)
3. [Modelowanie - scikit-learn](#scikit)
4. [Modelowanie - statsmodels](#stats)

***
***

# <a id='train_test'>Podział na set testowy i treningowy</a> 

1. Dokonaj podziału na set testowy i treningowy (test = 30% zbioru). Co jest wynikiem tej funkcji? Jaki typ danych i jakiego rozmiaru?

2. Ponownie dokonaj podziału, tym razem przypisując je do zmiennych df_train i df_test. Sprawdź, czy zgadzają się wielkości zbiorów?

3. Sprawdź jakie są rozkłady zmiennych wyjaśnianych w podzielonych zbiorach - czy wyszły równe?

4. Dokonaj podziału proporcjonalnie do zmiennej czy_szkoda. Sprawdź ponownie rozkłady

# <a id='encode'>Przekształcenia zmiennych kategorialnych</a> 

1. Stwórz obiekt ohe_enc będący obiektem kodowania one-hot_encoding z wybranymi parametrami (np. usunięcie pierwszej kategorii). 

2. Dopasuj go do zmiennych kategorialnych df_train

3. Wyświetl kategorie powstały po dopasowaniu

4. Wyświetl nazwy kolumn na podstawie których stworzono te kategorie

5. Wyświetl parametry dopasowania

6. Wyświetl, które kategorie reprezentują kolejne kolumny

7. Przekształć koder na zmiennych kategorialnych df_train i df_test tworząc X_train_ohe i X_test_ohe

8. Wyświetl df_train_ohe i df_test_ohe. Jaki to typ danych? Co należałoby zrobić, aby przekształcić je ponownie w dataframe?

9. Wyświetl nazwy poszczególnych kolumn X_train_ohe

10. Stwórz dataframe z kodowaniem OHE osobno dla próbki testowej i treningowej z poprawnymi nazwami kolumn

10. Powtórz kroki 1-10 dla kodowania porządkowego (ordinal encoding)

11. Analogicznie przeprowadź wybrane skalowanie zmiennych ciągłych (min-max lub standard). <br> UWAGA! Tylko dla zmiennych wyjaśniających!!!

12. Scal dataframe wybranej metody kodowania zmiennych kategorialnych i skalowania zniennych ciągłych (osobno dla próbki testowej i treningowej)

# <a id='scikit'>Modelowanie - scikit-learn</a> 

1. Dopasuj regresję Poissona do zmiennej wyjaśnianej 'czy_szkoda" na ustawieniach domyślnych

2. Wywietl współczyniki regresji dopasowania

3. Wyświetl nazwy poszczególnych zmiennych

4. Wyświetl stałą modelu

5. Stwórz tabelę zestawiającą punkt 2 z nazwami z punktu 3 oraz ze stałą modelu

6. Sprawdź dokładność dopasowania przy użyciu metryki standardowej oraz przy użyciu mean_poisson_deviance. Porównaj z modelem średniej (dummy)

7. Powtórz kroki 1-6 dla modelu średniej szkody oraz GammaRegressor (pamiętaj o wymaskowaniu wierszy, dla których nie było szkody)

# <a id='stats'>Modelowanie - statsmodels</a> 

1. Stwórz tekstową reprezentację równania regresji Poissona, które ma zostać rozwiązane

2. Stwórz reprezentacje dmatrix zmiennej wyjaśnianej czy_szkoda i zmiennych wyjaśniających dla próbki testowej i treningowej

3. Dokonaj przekształcenia zmiennych ciągłych zgodnie z wybranym skalowaniem na próbkach testowych i treningowych. Dołącz kolumnę 'EXPO' z samymi jedynakmi do każdego ze zbiorów

4. Rozwiąż równanie regresji Poissona w statsmodel (bez regularyzacji) i wyświetl wyniki

5. Sprawdź dopasowanie modelu na próbce testowej: dokonaj predykcji na podstawie modelu regresji. Policz wybrane miary dopasowania i porównaj z modelem scikit-lean

6. Powtórz kroki 4 i 5 dla wersji z regularyzacją i parametrami alpha=0.05 L1_wt=1.0

7. Powtórz kroki 1-6 dla modelu średniej szkody oraz GammaRegressor (pamiętaj o wymaskowaniu wierszy, dla których nie było szkody)